<a href="https://colab.research.google.com/github/deep569/bengali_question_answering_model-and-tokenizer_training/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install accelerate

In [ ]:
!mkdir ok


In [ ]:
!pip install accelerate

In [ ]:
!pip install -U PyPDF2
!pip install python-docx

In [ ]:
!pip install transformers

In [ ]:
!mkdir my

In [ ]:
import pandas as pd
import numpy as np
import re
from PyPDF2 import PdfReader
import os
import docx

In [ ]:
from datasets import load_dataset

raw_datasets = load_dataset("csebuetnlp/BanglaNMT")

In [ ]:
! pip install datasets transformers[sentencepiece]

In [ ]:
from datasets import load_dataset

# Replace 'dataset_name' with the name of your dataset
dataset_name = "csebuetnlp/BanglaNMT"

dataset = load_dataset(dataset_name)

keys = list(dataset["train"][0].keys())

print(keys)

In [ ]:
def get_training_corpus():
    dataset = raw_datasets["train"]
    for start_idx in range(0, len(dataset), 1000):
        samples = dataset[start_idx : start_idx + 1000]
        yield samples["bn"]

In [ ]:
from transformers import AutoTokenizer

old_tokenizer = AutoTokenizer.from_pretrained("gpt2")
training_corpus = get_training_corpus()
new_tokenizer = old_tokenizer.train_new_from_iterator(training_corpus,52000)
new_tokenizer.save_pretrained("")#save the new tokanizer with a new name in your local directory and load the newly trained tokanizer for training the model

In [ ]:
example = "গল্পটিতে"

print(old_tokenizer.tokenize(example))
print(new_tokenizer.tokenize(example))

In [ ]:
import torch
import torch.optim as optim
import torch.nn as nn

# Define your model (example)
class YourModel(nn.Module):
    def __init__(self):
        super(YourModel, self).__init__()
        self.fc1 = nn.Linear(100, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = YourModel()

optimizer = optim.AdamW(model.parameters(), lr=0.001)

num_epochs = 80


for epoch in range(num_epochs):

    ...


In [ ]:
# Functions to read different file types
def read_pdf(file_path):
    with open(file_path, "rb") as file:
        pdf_reader = PdfReader(file)
        text = ""
        for page_num in range(len(pdf_reader.pages)):
            text += pdf_reader.pages[page_num].extract_text()
    return text

def read_word(file_path):
    doc = docx.Document(file_path)
    text = ""
    for paragraph in doc.paragraphs:
        text += paragraph.text + "\n"
    return text

def read_txt(file_path):
    with open(file_path, "r") as file:
        text = file.read()
    return text

def read_documents_from_directory(directory):
    combined_text = ""
    for filename in os.listdir(directory):
        file_path = os.path.join(directory, filename)
        if filename.endswith(".pdf"):
            combined_text += read_pdf(file_path)
        elif filename.endswith(".docx"):
            combined_text += read_word(file_path)
        elif filename.endswith(".txt"):
            combined_text += read_txt(file_path)
    return combined_text


In [ ]:

train_directory = '/content/my'
text_data = read_documents_from_directory(train_directory)
text_data = re.sub(r'\n+', '\n', text_data).strip()  # Remove excess newline characters


In [ ]:
with open("/content/my/train.txt", "w") as f:
    f.write(text_data)

In [ ]:
!pip install wandb #install wandb

In [ ]:
# BengaliTokenizer
# AutoTokenizer

from transformers import AutoTokenizer, GPT2DoubleHeadsModel, TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments
from transformers import EvalPrediction
import torch
import csv



def load_dataset(train_file_path, val_file_path, tokenizer, block_size=128):
    train_dataset = TextDataset(
        tokenizer=tokenizer,
        file_path=train_file_path,
        block_size=block_size,
    )
    val_dataset = TextDataset(
        tokenizer=tokenizer,
        file_path=val_file_path,
        block_size=block_size,
    )
    return train_dataset, val_dataset
def load_data_collator(tokenizer, mlm=False):
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer,
        mlm=mlm,
    )
    return data_collator  # Return the same data collator for both train and validation

import csv
import torch

def compute_accuracy(labels_ids, preds_ids):
    accuracy = (preds_ids == labels_ids).mean()
    return accuracy.item()

def compute_metrics(p: EvalPrediction):
    labels_ids = p.label_ids
    preds_ids = p.predictions.argmax(-1)
    accuracy = compute_accuracy(labels_ids, preds_ids)

    # Save the labels, predictions, and accuracy to a CSV file
    with open("accuracy.csv", "w", newline='') as csvfile:
        writer = csv.writer(csvfile, delimiter=",")
        writer.writerow(["Labels", "Predictions", "Accuracy"])
        for label, prediction in zip(labels_ids, preds_ids):
            writer.writerow([label.tolist(), prediction.tolist(), accuracy])

    return {"accuracy": accuracy}



def train(train_file_path, val_file_path, model_name,
          output_dir,
          overwrite_output_dir,
          per_device_train_batch_size,
          num_train_epochs,
          save_steps, learning_rate):

    tokenizer = AutoTokenizer.from_pretrained("Koustav_Tokenizer")
    train_dataset, val_dataset = load_dataset(train_file_path, val_file_path, tokenizer)
    data_collator = load_data_collator(tokenizer)  # Use the same data collator for both training and validation

    tokenizer.save_pretrained(output_dir)

    model = GPT2DoubleHeadsModel.from_pretrained(model_name)

    model.save_pretrained(output_dir)

    training_args = TrainingArguments(
        output_dir=output_dir,
        overwrite_output_dir=overwrite_output_dir,
        per_device_train_batch_size=per_device_train_batch_size,
        num_train_epochs=num_train_epochs,
        learning_rate=learning_rate,
        evaluation_strategy="steps",  # Evaluate at specified steps
        eval_steps=save_steps,
        logging_steps=20, # Evaluate every `save_steps` during training
    )

    import csv




    trainer = Trainer(
        model=model,
        args=training_args,
        data_collator=data_collator,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,  # Set the validation dataset
        compute_metrics=compute_metrics,
       # Set the custom compute_metrics function
    )

    trainer.train()
    trainer.save_model()





train_file_path = "/content/my/train.txt"
val_file_path = "/content/my/val.txt"
model_name = 'gpt2-medium'
output_dir = '/content/my'
overwrite_output_dir = False
per_device_train_batch_size = 16
num_train_epochs = 100
save_steps = 20

# Add a parameter to adjust the learning rate
learning_rate = float(input("Enter the learning rate: "))

train(
    train_file_path=train_file_path,
    val_file_path=val_file_path,
    model_name=model_name,
    output_dir=output_dir,
    overwrite_output_dir=overwrite_output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    num_train_epochs=num_train_epochs,
    save_steps=save_steps,
    learning_rate=learning_rate,
)


Inference

In [ ]:
from transformers import AutoTokenizer, GPT2LMHeadModel, TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments
from transformers import EvalPrediction
import csv



In [ ]:
def load_model(model_path):
    model = GPT2DoubleHeadsModel.from_pretrained(model_path)
    return model


def load_tokenizer(tokenizer_path):
    tokenizer = AutoTokenizer.from_pretrained("Koustav_Tokenizer")
    return tokenizer

def generate_text(model_path, prompt, max_new_tokens=50):
    model = load_model(model_path)
    tokenizer = load_tokenizer(model_path)
    ids = tokenizer.encode(f'{prompt}', return_tensors='pt')
    # Set max_length to a very large value to allow unrestricted text generation
    final_outputs = model.generate(
        ids,
        do_sample=True,
        max_new_tokens=max_new_tokens,
        pad_token_id=model.config.eos_token_id,
        top_k=50,
        top_p=0.95,
        temperature=0.5,
        early_stopping=True,  # Prevent the model from generating more text after the first end-of-sequence token
         num_return_sequences=1,
    )
    print(tokenizer.decode(final_outputs[0], skip_special_tokens=True))



This model got trained on the entire text and took much longer to train, and yet it fails to give meaningful results.

In [ ]:
model1_path = "/content/my" #load your saved model

prompt= "হ্যালো, আপনি কেমন আছেন:"
max_new_tokens=100

generate_text(model1_path,prompt, max_new_tokens)

হ্যালো, আপনি কেমন আছেন: ফোনিয়ার প্রাধ্যক্ষ ড. জিয়া রহমান বিষয়টি নিশ্চিত করেন।"	
"Shafiqul Islam, Chief of Criminal Investigation Departmen


In [ ]:
import torch
import torch.nn.functional as F
from transformers import GPT2DoubleHeadsModel, AutoTokenizer
import csv

def calculate_accuracy(logits, targets):
    predicted_tokens = logits.argmax(dim=-1)
    accuracy = (predicted_tokens == targets).float().mean()
    return accuracy

if __name__ == "__main__":
    # Load your trained GPT-2 model and tokenizer (change the paths accordingly)
    model_path = "/content/my"
    tokenizer_path = ""#enter the path of the newly trained tokenizer
    model = GPT2DoubleHeadsModel.from_pretrained(model_path)
    tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)

    # Initialize a list to store user inputs and their respective accuracies
    user_inputs = []
    accuracies = []

    # Take at least 50 user inputs
    while len(user_inputs) < 2:
        user_input = input("Enter your text: ")
        if user_input.strip():
            user_inputs.append(user_input)

            # Tokenize the user input and return PyTorch tensors
            test_inputs = tokenizer([user_input], return_tensors="pt", truncation=True)

            with torch.no_grad():
                model.eval()
                input_ids = test_inputs["input_ids"].unsqueeze(0)
                outputs = model(input_ids)[0]

                predicted_probs = F.softmax(outputs, dim=-1)

                predicted_tokens = predicted_probs.argmax(dim=-1).squeeze()

                target_tokens = input_ids.squeeze()

                accuracy = calculate_accuracy(predicted_tokens, target_tokens)

                accuracies.append(accuracy.item())

    # Save the user inputs and accuracies to a CSV file
    with open("user_accuracies.csv", "w", newline="") as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(["User Input", "Accuracy"])
        for user_input, accuracy in zip(user_inputs, accuracies):
            writer.writerow([user_input, accuracy])

    print("User inputs and accuracies have been saved to 'user_accuracies.csv'.")


Enter your text: পোরকুপাইনরা কী খায়	
Enter your text: exit
User inputs and accuracies have been saved to 'user_accuracies.csv'.
